In [ ]:

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## WHAT WE FOUND WHILE EXPLORING THE DATA:\n",
    "\n",
    "- We need to apply a proper schema\n",
    "- The date column needs fixing\n",
    "- We need to extract/remove usernames\n",
    "- We need to extract hashtags and replace them with their word equivalent\n",
    "- We need to remove URLs, as our algorithm won't need that or use that\n",
    "- The same goes for email-addresses\n",
    "- Symbols stored in HTML notation do not appear properly unescaped (example: &lt;3 and s&amp;^t)\n",
    "- Unwanted characters are present, perhaps  Encoding is ‘broken’ ? (example: �����ߧ�ǿ�����ж�؜��� )\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 56,
   "metadata": {},
   "outputs": [],
   "source": [
    "import html\n",
    "import pandas as pd\n",
    "from pyspark.sql import SparkSession\n",
    "from pyspark.sql import functions as f\n",
    "\n",
    "spark = SparkSession.builder.appName(\"DataCleaning\").getOrCreate()\n",
    "\n",
    "pd.options.display.max_columns = None\n",
    "pd.options.display.max_rows = 250\n",
    "pd.options.display.max_colwidth = 150\n",
    "\n",
    "schema = \"polarity FLOAT, id LONG, date_time TIMESTAMP, query STRING, user STRING, text STRING\"\n",
    "timestampformat = \"EEE MMM dd HH:mm:ss zzz yyyy\"\n",
    "\n",
    "IN_PATH = \"/home/jovyan/data-sets/sentiment-140-training-data/RAW\"\n",
    "OUT_PATH = \"/home/jovyan/data-sets/sentiment-140-training-data/CLEAN\"\n",
    "\n",
    "spark_reader = spark.read.schema(schema)\n",
    "\n",
    "\n",
    "@f.udf\n",
    "def html_unescape(s: str):\n",
    "    if isinstance(s, str):\n",
    "        return html.unescape(s)\n",
    "    return s\n",
    "\n",
    "\n",
    "def clean_data(df):\n",
    "    df = (\n",
    "        df\n",
    "        .withColumn(\"original_text\", f.col(\"text\"))\n",
    "        .withColumn(\"text\", f.regexp_replace(f.col(\"text\"), url_regex, \"\"))\n",
    "        .withColumn(\"text\", f.regexp_replace(f.col(\"text\"), email_regex, \"\"))\n",
    "        .withColumn(\"text\", f.regexp_replace(f.col(\"text\"), user_regex, \"\"))\n",
    "        .withColumn(\"text\", f.regexp_replace(f.col(\"text\"), \"#\", \" \"))\n",
    "        .withColumn(\"text\", html_unescape(f.col(\"text\")))\n",
    "        .filter(\"text != ''\")\n",
    "    )\n",
    "    return df\n",
    "\n",
    "df_raw = spark_reader.csv(IN_PATH, timestampFormat=timestampformat) \n",
    "df_clean = clean_data(df_raw)\n",
    "\n",
    "df_clean.write.partitionBy(\"polarity\").parquet(OUT_PATH, mode=\"overwrite\")\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
